In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
import sys
from pathlib import Path
import anndata as ad
import numpy as np
# import dynamo as dyn
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

sys.path.insert(0, "../../scripts/methods/")
from my_moscot import moscot_align, moscot_align_ref

sys.path.insert(0, "/home/ylu/project")
from utils import *
from tqdm import tqdm

In [2]:
data_folder = "../../data/STARMapPlus_nonrigid_simulation_different_distort/"

method = 'Moscot_N'
results_folder = "./nonrigid_different_distort_results/"+method
Path(results_folder).mkdir(parents=True, exist_ok=True)


In [3]:
slice_name = ['A','B','C','D']
tps_slice_name = ['A*','B*','C*','D*']
i = 10
data_name = "well_{}".format(i)
s = ad.read_h5ad('../../data/STARMapPlus/{}.h5ad'.format(data_name))
norm_scale = np.linalg.norm(s.obsm['spatial'].max(0) - s.obsm['spatial'].min(0))
del s
j = 1
slice1 = ad.read_h5ad(os.path.join(data_folder, data_name, "slice_" + str(j) + ".h5ad"))
slice2 = ad.read_h5ad(os.path.join(data_folder, data_name, "slice_" + str(j+1) + ".h5ad"))
repeat = 10

In [5]:
for l, no in enumerate(np.arange(0,0.22,0.02)):
    alignment_results = []
    for k in range(repeat):
        slice1.obsm['prealign_spatial'] = slice1.obsm['spatial'].copy()
        slice2.obsm['prealign_spatial'] = slice2.obsm['tps_spatial_{}_r_{}'.format(l, k)].copy()
        align_slices, pis = moscot_align(
            models=[slice1, slice2],
            spatial_key="prealign_spatial",
            key_added="align_spatial",
            device="cuda",
            verbose=False,
            mode='warp',
            alpha=0.5,
        )
        
        torch.cuda.empty_cache()
        mae, mae_var = MAE(align_slices[1].obsm['align_spatial'], align_slices[1].obsm['spatial']) / norm_scale
        mse, mse_var = MSE(align_slices[1].obsm['align_spatial'], align_slices[1].obsm['spatial']) / norm_scale**2
        print(f'------iteration: {l}----repeat: {k}-----mae: {mae}---------')
        
        alignment_results.append([mae, mae_var, mse, mse_var, data_name, no, l, k])

    np.save(os.path.join(results_folder, data_name + f"_{method}_differ_distort_{l}.npy"), alignment_results, allow_pickle=True)

INFO     Computing pca with `n_comps=30` for `xy` using `adata.X`                                                  
INFO     Normalizing spatial coordinates of `x`.                                                                   
INFO     Normalizing spatial coordinates of `y`.                                                                   
INFO     Solving `1` problems                                                                                      
INFO     Solving problem OTProblem[stage='prepared', shape=(11317, 11317)].                                        
WARNING  Solver did not converge                                                                                   
------iteration: 0----repeat: 0-----mae: 0.07457302181398734---------
INFO     Computing pca with `n_comps=30` for `xy` using `adata.X`                                                  
INFO     Normalizing spatial coordinates of `x`.                                                                   
IN